# Time Series Counterfactual Analysis

**Goal**: Analyze historical "what-if" scenarios - what would have happened if we intervened earlier?

This notebook demonstrates:
1. Historical counterfactual analysis ("What if we intervened 30 days ago?")
2. Time series intervention impact assessment
3. Optimal intervention timing analysis
4. Comparing actual outcomes vs. counterfactual predictions

## 1. Setup and Data Loading

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Load time series data
df = pd.read_csv('data/retail_data.csv')
print(f"Loaded {len(df)} retail stores")
df.head()

## 2. Define Causal Graph

In [ ]:
# Simplified causal structure for time series analysis
nodes = ['marketing_spend', 'price_discount', 'staff_count', 
         'foot_traffic', 'conversion_rate', 'customer_satisfaction', 'sales']

edges = [
    ('marketing_spend', 'foot_traffic'),
    ('price_discount', 'conversion_rate'),
    ('staff_count', 'customer_satisfaction'),
    ('foot_traffic', 'sales'),
    ('conversion_rate', 'sales'),
    ('customer_satisfaction', 'sales')
]

# Create adjacency matrix
adj_matrix = pd.DataFrame(0, index=nodes, columns=nodes)
for parent, child in edges:
    adj_matrix.loc[parent, child] = 1

print("Causal Graph (Time Series):")
print(adj_matrix)

## 3. Train Causal Model

In [ ]:
from ht_categ import HT, HTConfig

# Train HT model
config = HTConfig(graph=adj_matrix, model_type='XGBoost')
ht_model = HT(config)
ht_model.train(df)

print("✓ Causal model trained")
print(f"\nModel Quality (R²):")
for node, metrics in ht_model.model_metrics.items():
    if 'r2_score' in metrics:
        print(f"  {node}: {metrics['r2_score']:.3f}")

## 4. Historical Counterfactual: "What if we intervened 30 days ago?"

In [ ]:
from intervention_search.visualization.timeseries_intervention import TimeSeriesInterventionAnalyzer

# Initialize time series analyzer with proper parameters
ts_analyzer = TimeSeriesInterventionAnalyzer(
    graph=ht_model.graph,
    regressors_dict=ht_model.regressors_dict,
    baseline_stats=ht_model.baseline_stats,
    node_types=ht_model.node_types,
    label_encoders=getattr(ht_model, 'label_encoders', None)
)

# Create sample time series data from historical observations
# For demonstration, we'll simulate 60 days of data
np.random.seed(42)
n_days = 60

# Generate historical time series data
historical_data = pd.DataFrame({
    'marketing_spend': np.random.normal(5000, 1000, n_days),
    'price_discount': np.random.normal(15, 3, n_days),
    'staff_count': np.random.randint(5, 15, n_days),
    'foot_traffic': np.random.normal(3000, 500, n_days),
    'conversion_rate': np.random.normal(25, 5, n_days),
    'customer_satisfaction': np.random.normal(70, 10, n_days),
    'sales': np.random.normal(80000, 15000, n_days)
})

# Simulate historical intervention
# Scenario: "What if we increased marketing spend by 25%, 30 days ago?"
result = ts_analyzer.simulate_historical_intervention(
    historical_data=historical_data,
    intervention_node='marketing_spend',
    intervention_pct_change=25.0,  # +25% increase
    outcome_node='sales',
    intervention_start_date=30  # 30 days ago (row index)
)

print("\n" + "="*70)
print("HISTORICAL COUNTERFACTUAL ANALYSIS")
print("="*70)
print(f"\nScenario: Increase marketing_spend by +25%, 30 days ago")
print(f"\nActual Average Sales (last 30 days): ${result.actual_outcome_series.iloc[30:].mean():,.2f}")
print(f"Counterfactual Sales (if intervened): ${result.counterfactual_outcome_series.iloc[30:].mean():,.2f}")
print(f"Cumulative Effect: ${result.cumulative_effect:,.2f}")
print(f"\nMissed Opportunity: {(result.cumulative_effect / result.actual_outcome_series.iloc[30:].sum() * 100):+.1f}%")
print("="*70)

## 5. Visualize Counterfactual vs. Actual

In [ ]:
# Plot comparison
fig, axes = ts_analyzer.plot_intervention_comparison(
    result=result,
    title='Counterfactual Analysis: Marketing Spend +25% (30 Days Ago)',
    figsize=(14, 6),
    show_intervention_line=True
)

plt.show()

## 6. Multi-Scenario Analysis: Compare Different Intervention Timings

In [ ]:
# Test different intervention timings
timings = [7, 14, 21, 30, 45]  # days ago (row indices)

timing_results = []

for days_ago in timings:
    result_timing = ts_analyzer.simulate_historical_intervention(
        historical_data=historical_data,
        intervention_node='marketing_spend',
        intervention_pct_change=25.0,
        outcome_node='sales',
        intervention_start_date=days_ago
    )
    
    # Calculate total potential gain from intervention date onwards
    total_gain = result_timing.cumulative_effect
    gain_pct = (total_gain / result_timing.actual_outcome_series.iloc[days_ago:].sum() * 100) if len(result_timing.actual_outcome_series.iloc[days_ago:]) > 0 else 0
    
    timing_results.append({
        'days_ago': days_ago,
        'gain_pct': gain_pct,
        'total_gain': total_gain
    })

# Display results
timing_df = pd.DataFrame(timing_results)
print("\n" + "="*70)
print("INTERVENTION TIMING ANALYSIS")
print("="*70)
print("\nOptimal Intervention Timing:")
print(timing_df.sort_values('total_gain', ascending=False))

# Plot
plt.figure(figsize=(10, 5))
plt.bar(timing_df['days_ago'], timing_df['total_gain'], color='steelblue', alpha=0.7)
plt.xlabel('Days Ago (Intervention Timing)')
plt.ylabel('Total Potential Gain ($)')
plt.title('Intervention Value by Timing')
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

best_timing_idx = timing_df['total_gain'].idxmax()
print(f"\n✅ Optimal timing: {timing_df.loc[best_timing_idx, 'days_ago']:.0f} days ago")
print(f"   Total gain: ${timing_df.loc[best_timing_idx, 'total_gain']:,.2f}")
print("="*70)

## 7. Forward-Looking Intervention Planning

In [ ]:
from intervention_search import InterventionSearch

# Find optimal intervention for FUTURE with increased simulations for narrower CIs
searcher = InterventionSearch(
    graph=ht_model.graph,
    ht_model=ht_model,
    n_simulations=5000  # Increased from 1000 for more precise confidence intervals
)

results = searcher.find_interventions(
    target_outcome='sales',
    target_change=20.0,  # +20% increase
    tolerance=3.0,
    confidence_level=0.90,
    verbose=True
)

best = results['best_intervention']

print("\n" + "="*70)
print("FORWARD-LOOKING RECOMMENDATION")
print("="*70)
print(f"\nRecommended Action NOW:")
print(f"  Variable: {best['nodes'][0]}")
print(f"  Change: {list(best['required_pct_changes'].values())[0]:+.1f}%")
print(f"\nExpected Impact (over next 30 days):")
print(f"  Sales Increase: {best['actual_effect']:+.1f}%")
print(f"  90% Confidence Interval: [{best['ci_90'][0]:+.1f}%, {best['ci_90'][1]:+.1f}%]")
print(f"  Confidence: {best['confidence']:.0%}")
print(f"  Status: {'✅ APPROVED' if best.get('within_tolerance', False) else '⚠️ NEEDS REVIEW'}")
print("="*70)

## 8. Export Counterfactual Data for Reporting

In [ ]:
# Export detailed counterfactual data
output_path = 'counterfactual_analysis_results.csv'

# Run the analysis to get the result
export_result = ts_analyzer.simulate_historical_intervention(
    historical_data=historical_data,
    intervention_node='marketing_spend',
    intervention_pct_change=25.0,
    outcome_node='sales',
    intervention_start_date=30
)

# Export to file
ts_analyzer.export_counterfactual_data(
    result=export_result,
    output_path=output_path,
    format='csv'
)

# Load and preview
counterfactual_data = pd.read_csv(output_path)
print(f"\nPreview:")
print(counterfactual_data.head(10))

## Summary

This notebook demonstrated:

**Historical Analysis:**
- ✅ Counterfactual predictions ("what if we intervened N days ago?")
- ✅ Quantified missed opportunities
- ✅ Optimal intervention timing analysis

**Forward Planning:**
- ✅ Future intervention recommendations
- ✅ Confidence intervals for time series predictions
- ✅ Quality-gated recommendations

**Business Value:**
- Learn from historical missed opportunities
- Optimize intervention timing
- Make data-driven decisions with uncertainty quantification
- Export results for executive reporting